In [1]:
import pandas as pd
import openpyxl

# Parsing Committee Assignment Data

Since various committee websites have unpredictable underlying HTML structure, I recommend that the team creates a spreadsheet of relevant info, and this notebook will bridge the gap from these spreadsheets to the database back-end of the LegTracker tool.

Outside of this notebook:
 - Download the manually created spreadsheet of committees (names with hyperlinks) and memberships
 - Connect to the database through terminal 
 - Download the `legislator` table from the database into the working directory
 - Upload the resulting CSVs into the database's `committee` and `committee_assignment` tables respectively

## Stage 1: Parse data for the committee table

In [2]:
def _get_link_if_exists(cell) -> str | None:
    try:
        return cell.hyperlink.target
    except AttributeError:
        return None

def extract_hyperlinks_from_xlsx(
    file_name: str, sheet_name: str, columns_to_parse: list[str], row_header: int = 1
) -> pd.DataFrame:
    df = pd.read_excel(file_name, sheet_name)
    ws = openpyxl.load_workbook(file_name)[sheet_name]
    for column in columns_to_parse:
        row_offset = row_header + 1
        column_index = list(df.columns).index(column) + 1
        df['webpage_link'] = [
            _get_link_if_exists(ws.cell(row=row_offset + i, column=column_index))
            for i in range(len(df[column]))
        ]
    return df

def parse_for_db(df):
    restructured = df.loc[:, ["Committee", "webpage_link"]]
    restructured.rename(columns={"Committee": "name"}, inplace=True)
    # CHANGE THE FOLLOWING LINE WITH A VALID CHAMBER ID
    restructured["chamber_id"] = 2 
    restructured = restructured[["chamber_id", "name", "webpage_link"]]
    restructured.to_csv('updated_committee_table.csv', index=False)

raw = extract_hyperlinks_from_xlsx(
    'senate_cmte_assignments.xlsx',
    'Sheet1', # CHANGE TO CORRECT SHEET NAME
    ['Committee']
)

parse_for_db(raw)

## Stage 2: Insert committee data and extract serial committee IDs

When transforming committee assignment data points, we need both the `legislator` table contents and (freshly) updated `committee` table contents. The combination allows us to create the `legislator_id` and `committee_id` pairing that defines an assignment.

Before running the next blocks of code, make sure to upsert the newly created committee data into the table. Then, download it to the working directory so we have access to the updated `committee_id` mapping for the following code.

In [3]:
db_committee = pd.read_csv('db_committee.csv')
db_committee = db_committee.loc[db_committee.chamber_id == 2, :] # CHANGE THIS CONDITIONAL FILTER TO THE CORRECT CHAMBER ID

In [4]:
db_legislator = pd.read_csv('db_legislator.csv')
db_legislator.rename(columns={'name': 'legislator_name'}, inplace=True)

## Stage 3: Parse Committee Assignments and Map to ID values

In [5]:
import re
def transform_name(name):
    name = name.strip()
    parts = re.split(r'\s+', name.replace(',', ''))
    no_title = len(name.split(",")) == 1
    if no_title:
        return f"{parts[-1]}, {' '.join(parts[0:-1])}"
    else:
        return f"{parts[-2]}, {' '.join(parts[0:-2])}, {parts[-1].replace(',', '')}"

In [9]:
# each row in the raw data is a committee, total length is N committees
# each element of the result list is a committee membership, total length is (at most) N committees * M members
# for each cell in that row, create a sublist of the target format [committee_name, chamber_id, legislator_name, assignment_type]
# if cell is column 0, set to "committee_name" value
# if cell is column 1 or 2, assignment_type is "chair" or "vice chair"
# if cell column >= 3, assignment_type is "member"

def parse_assignments(df, chamber_id):
    result = list()
    for i in range(len(df)):
        committee_name = df.iloc[i, 0]
        for j in range(1, len(df.columns)):
            cell = df.iloc[i, j]
            if type(cell) == str: 
                if ".gov" in cell:
                    continue
                else:
                    curr = [committee_name, chamber_id, transform_name(cell)] 
                    if j == 1:
                        curr.append("Chair")
                    elif j == 2:
                        curr.append("Vice Chair")
                    elif "Member" in df.columns[j]:
                        curr.append("Member")
                    result.append(curr)
            else:
                continue
    return pd.DataFrame(result, columns=['name', 'chamber_id', 'legislator_name', 'assignment_type'])

assignments = parse_assignments(raw, 2)

## Stage 4: Identify missed mappings

Some legislators use variations on their full names on committee websites, or colloquially (ex: James Doe may go by Jim Doe among his staffers, who use the name "Jim" on his committee profile). This means that our mapping will not be perfect with our current database limitations. We need to find faulty assignment data points and manually add them to the set to be inserted into the database.

In [19]:
def map_committee_id(df, committee_table):
    validate = [len(df), 0]
    mapped = (df.merge(committee_table.loc[:, ['committee_id', 'name']], on='name')
              .reindex(columns=['committee_id', 'name', 'chamber_id', 'legislator_name', 'assignment_type']))
    validate[1] = len(mapped)
    if validate[0] != validate[1]:
        diff = max(validate) - min(validate)
        print(f"WARNING: {diff} ASSIGNMENTS COULD NOT BE MAPPED. CHECK THAT *COMMITTEE* NAMES ARE CONSISTENT")
        return df, mapped
    else:
        return mapped

def map_legislator_id(df, legislator_table):
    validate = [len(df), 0]
    mapped = (df.merge(legislator_table.loc[:,['legislator_id', 'legislator_name']], on='legislator_name')
              .reindex(columns=['committee_id', 'name', 'chamber_id', 'legislator_name', 'assignment_type', 'legislator_id']))
    validate[1] = len(mapped)
    if validate[0] != validate[1]:
        diff = max(validate) - min(validate)
        print(f"WARNING: {diff} ASSIGNMENTS COULD NOT BE MAPPED. CHECK THAT *LEGISLATOR* NAMES ARE CONSISTENT")
        return df, mapped
    else:
        return mapped

def find_faulty_rows(before, after, column_name):
    lost = before[~before[column_name].isin(after[column_name])]
    added = after[~after[column_name].isin(before[column_name])]
    if len(lost):
        print(f"{len(lost)} rows lost")
        print("#"*60)
        print(lost)
    if len(added):
        print(f"{len(added)} rows added")
        print(added)
    return

def map_values(df, committee_table, legislator_table):
    temp = map_committee_id(df, db_committee)
    if type(temp) == tuple:
        find_faulty_rows(temp[0], temp[1], 'name')
        return temp[1]
    result = map_legislator_id(temp, db_legislator)
    if type(result) == tuple:
        find_faulty_rows(result[0], result[1], 'legislator_name')
        return result # whatever is missing can be added after manual discovery
    return result

mapped = map_values(assignments, db_committee, db_legislator)

3 rows lost
############################################################
     committee_id                      name  chamber_id      legislator_name  \
33             50  Budget and Fiscal Review           2  Padilla, Stephen C.   
34             50  Budget and Fiscal Review           2        Roth, Richard   
193            68                     Rules           2   Eggman, Talamantes   

    assignment_type  
33           Member  
34           Member  
193          Member  


In [18]:
# edit 'contains' argument to manually discover legislator ID
db_legislator.loc[db_legislator.legislator_name.str.contains("Eggman"),:]

,legislator_id,chamber_id,legislator_name,district,party
34,35,2,"Eggman, Susan Talamantes",5,Democratic


In [ ]:
# insert assignments.loc[INDEX] and [LEGISLATOR_ID_NUMBER] manually at the end of the final DF
before = mapped[0]
final_df = mapped[1]
final_df.loc[len(final_df)] = mapped[0].loc[193].tolist() + [35]

In [ ]:
# drop committee name, legislator name to match Lucid
final_df = final_df.loc[:, ['legislator_id', 'committee_id', 'assignment_type']]

In [ ]:
final_df.to_csv('committee_assignments.csv', index=False)